In [8]:
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print("Eğitim verisi boyutu:", train.shape)
print("Test verisi boyutu:", test.shape)
print("\nilk 5 satir:")
train.head()

Eğitim verisi boyutu: (8693, 14)
Test verisi boyutu: (4277, 13)

ilk 5 satir:


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [10]:
print("Sutunlar:", train.columns.tolist())
print("\neksik degerler:")
print(train.isnull().sum())
print("\nhedef degisken dagilimi:")
print(train['Transported'].value_counts())

Sutunlar: ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported']

eksik degerler:
PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

hedef degisken dagilimi:
Transported
True     4378
False    4315
Name: count, dtype: int64


In [11]:
y_train = train['Transported']

train_ids = train['PassengerId']
test_ids = test['PassengerId']

feature_cols = ['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

X_train = train[feature_cols].copy()
X_test = test[feature_cols].copy()

print("X_train boyutu:", X_train.shape)
print("X_test boyutu:", X_test.shape)


X_train boyutu: (8693, 10)
X_test boyutu: (4277, 10)


In [12]:
numeric_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
X_train[numeric_cols] = X_train[numeric_cols].fillna(0)
X_test[numeric_cols] = X_test[numeric_cols].fillna(0)

categorical_cols = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']
X_train[categorical_cols] = X_train[categorical_cols].fillna('Unknown')
X_test[categorical_cols] = X_test[categorical_cols].fillna('Unknown')

print("eksik degerler kaldi mi?", X_train.isnull().sum().any)

eksik degerler kaldi mi? <bound method Series.any of HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
dtype: int64>


In [20]:
from sklearn.preprocessing import LabelEncoder

for col in categorical_cols:
    le = LabelEncoder()

    # Ensure all values are strings before concatenation and encoding
    X_train[col] = X_train[col].astype(str)
    X_test[col] = X_test[col].astype(str)

    combined = pd.concat([X_train[col], X_test[col]])
    le.fit(combined)

    X_train[col] = le.transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

print("Kategorik degiskenler sayisala cevrildi")
print(X_train.head())

Kategorik degiskenler sayisala cevrildi
   HomePlanet  CryoSleep  Destination   Age  VIP  RoomService  FoodCourt  \
0           1          0            2  39.0    0          0.0        0.0   
1           0          0            2  24.0    0        109.0        9.0   
2           1          0            2  58.0    1         43.0     3576.0   
3           1          0            2  33.0    0          0.0     1283.0   
4           0          0            2  16.0    0        303.0       70.0   

   ShoppingMall     Spa  VRDeck  
0           0.0     0.0     0.0  
1          25.0   549.0    44.0  
2           0.0  6715.0    49.0  
3         371.0  3329.0   193.0  
4         151.0   565.0     2.0  


In [21]:
import xgboost as xgb

model = xgb.XGBClassifier(n_estimators = 100, max_depth = 5, learning_rate = 0.1,random_state = 42)

print("model egitiliyor...")
model.fit(X_train, y_train)
print("model egitimi tamamlandi")

train_accuracy = model.score(X_train, y_train)
print(f"Egitim Dogruluk Orani: {train_accuracy:.4f}")

model egitiliyor...
model egitimi tamamlandi
Egitim Dogruluk Orani: 0.8222


In [22]:
predictions = model.predict(X_test)

predictions_bool = predictions.astype(bool)

print(f"Toplam tahmin: {len(predictions)}")
print(f"True tahminler: {predictions_bool.sum()}")
print(f"False tahminler: {(~predictions_bool).sum()}")

Toplam tahmin: 4277
True tahminler: 2309
False tahminler: 1968


In [23]:
submission = pd.DataFrame({
    'PassengerId': test_ids,
    'Transported': predictions_bool
})


submission.to_csv('my_first_submission.csv', index=False)

print("Submission dosyasi olusturuldu!")
print(submission.head())

Submission dosyasi olusturuldu!
  PassengerId  Transported
0     0013_01         True
1     0018_01        False
2     0019_01         True
3     0021_01         True
4     0023_01         True
